# 4. OOP in Python - Part 2

This notebook assumes you can already create and use Python classes. In this notebook, we'll take a deeper look at classes.

## 4.1 Multiple Inheritance

In Python, it's possible to do multiple inheritance.

The new class inherits from all derived classes, __from left to right__, subject to the method resolution order. The MRO uses a linearization algorithm that is stable and consistent.

The importance of the method resolution order becomes more apparent when dealing with multiple inheritance as well as when using of the `super()` function.

Let's create a new `ThreeDimensionSquare` that derives from both `ThreeDimension` and `Square`. First we'll make a new `ThreeDimension` class that will make our shapes three-dimensional.

In [ ]:
# We've added all the necessary classes from the previous notebook.

class Shape(object):
    """Base class for shapes."""

    x = 0
    y = 0
    
    def width(self):
        """Retun width or equivalent."""
        return self.x
    
    def height(self):
        """Retun height or equivalent."""
        return self.y
    
    def area(self):
        return self.x * self.y


class Square(Shape):
    """Square implementation of Shape."""
    
    def __init__(self, **kwargs):
        self.x = kwargs.get('side', 1)
        self.y = self.x
    
    def __str__(self):
        return "{0} ({1}x{1})".format(self.__class__.__name__, self.x)

    @staticmethod
    def help(*args, **kwargs):
        print("Help for Square class: Accepts side and int value as keyword argument.")
        return args, kwargs

    @classmethod
    def side(cls, side):
        return cls(side=side)

    @property
    def X(self):
        return self.x

#    @X.setter
#    def X(self, value):
#        self.x = value

    @X.deleter
    def X(self):
        del self._x


class ThreeDimension(object):
    """Makes things 3D!"""
    
    z = 0

    def __init__(self, **kwargs):
        self.z = kwargs.get('depth', 1)
        super().__init__(**kwargs)

    def volume(self):
        """Because 3d!"""
        return self.x * self.y * self.z


class ThreeDimensionSquare(ThreeDimension, Square):
    """A 3D square or a cube."""

    def __init__(self, **kwargs):
        """Because it can be a cube, we can accept either depth or use side."""
        self.z = kwargs.get('depth') or kwargs.get('side', 1)
        super(ThreeDimension, self).__init__(**kwargs)


cube = ThreeDimensionSquare(side=2)
cube.volume()

## 4.2 [Method Resolution Order](https://docs.python.org/3.5/glossary.html#term-method-resolution-order)

Since `Square` is derived from `Shape`, note the __method resolution order__ section when the `cube` instance was passed as an argument to `help()`. Method resolution order or MRO determines how our object will look for information. Let's say my class derives from a tall chain of classes and/or from multiple clases, with some overrides along the way in different places, the MRO algorithm can give us a stable and consistent order of classes for looking up attributes.

In [ ]:
help(cube)

```
class ThreeDimensionSquare(ThreeDimension, Square)
 |  A 3D square or a cube.
 |  
 |  Method resolution order:
 |      ThreeDimensionSquare
 |      ThreeDimension
 |      Square
 |      Shape
 |      builtins.object
```

Each class besides `Shape` overrides the `__init__` method. Which `__init__` method is executed? The method resolution order determines this. The top-most class is the current class. It will search each one going down. Notice that the one at the bottom is `builtins.object` or the `object` subclassed in "new style" classes.

## 4.3 Super

There are times when we want to access the parent or ancestor's method before it was overridden. We might want to maintain the original functionality, add or modify it, but not rewrite everything. The `super()` function allows us to do exactly this. Consider the `__init__` method in our __`ThreeDimensionSquare`__ class:

In [ ]:
class ThreeDimensionSquare(ThreeDimension, Square):
    """A 3D square or a cube."""

    def __init__(self, **kwargs):
        """Because it can be a cube, we can accept either depth or use side."""
        self.z = kwargs.get('depth') or kwargs.get('side', 1)
        super(ThreeDimension, self).__init__(**kwargs)  # this

cube = ThreeDimensionSquare(side=2)
cube.volume()

One way to think of it is that when `super()` is called, is passes through to the parent or ancestor. In `ThreeDimensionSquare.__init__`, the value of `self.z` is set in a different way. The next line means to execute the `__init__` that was inherited by `ThreeDimension`, not the one that is defined in this class (`ThreeDimensionSquare`) or the one in `ThreeDimension` itself.

Let's change it a bit so we can see differences depending on how it's used. We'll change the statement to:

```
super(ThreeDimensionSquare, self).__init__(**kwargs)
```

So the argument to `super()` is the current class. Notice the output of `cube.column()` change to 4. The reason for this is because the `__init__` being executed is the one from `ThreeDimension` which has the line:

```
self.z = kwargs.get('depth') or kwargs.get('side', 1)
```

In [ ]:
class ThreeDimensionSquare(ThreeDimension, Square):
    """A 3D square or a cube."""

    def __init__(self, **kwargs):
        """Because it can be a cube, we can accept either depth or use side."""
        self.z = kwargs.get('depth') or kwargs.get('side', 1)
        super(ThreeDimensionSquare, self).__init__(**kwargs)  # this

cube = ThreeDimensionSquare(side=2)
cube.volume()

The value of `self.z` is changed to 1 so the output from `cube.volume()` is different.

Note that in Python 3, base class and type parameters in `super()` are optional. `super(ThreeDimensionSquare, self).__init__(**kwargs)` achieves the same effect as `super().__init__(**kwargs)` in the `ThreeDimensionSquare` class.

In [ ]:
class ThreeDimensionSquare(ThreeDimension, Square):
    """A 3D square or a cube."""

    def __init__(self, **kwargs):
        """Because it can be a cube, we can accept either depth or use side."""
        self.z = kwargs.get('depth') or kwargs.get('side', 1)
        super().__init__(**kwargs)  # this

cube = ThreeDimensionSquare(side=2)
cube.volume()

The `super()` function is not restricted to your current object. You can use super on any of your object's derived classes and on their methods, just like in our original `ThreeDimentionSquare` class.

In the next example, note how we are using super, passing `Square` as the argument. `BrokenShape.__init__()` is executed the execution is passed to next class in the hierarchy, which is `BrokenShape`.

In [ ]:
class BrokenShape(object):
    
    def __init__(self, **kwargs):
        print('I broke it.')

class BrokenThreeDimensionSquare(ThreeDimensionSquare, BrokenShape):

    def __init__(self, **kwargs):
        super(Square, self).__init__(**kwargs)


xcube = BrokenThreeDimensionSquare(side=2)
xcube.volume()

In [ ]:
help(xcube)

All our derived classes override `__init__()`. The method resolution order dictates that the `__init__()` that will execute is the one on the left most. But we made it so that the `__init__()` on the right most class is executed using `super()` ang passing a class that is further down the order. The `Shape` class doesn't override `__init__()` so the next class, `BrokenShape` is searched. It finds `__init__()` there and executes it.

```
class BrokenThreeDimensionSquare(ThreeDimensionSquare, BrokenShape)
 |  A 3D square or a cube.
 |  
 |  Method resolution order:
 |      BrokenThreeDimensionSquare
 |      ThreeDimensionSquare
 |      ThreeDimension
 |      Square
 |      Shape
 |      BrokenShape
 |      builtins.object
```


## 4.4 Static methods, Class methods, and Properties

### [Static methods](https://docs.python.org/3.5/library/functions.html#staticmethod)

> A static method does not receive an implicit first argument (like `self`). It can be called either on the class (such as Class.method()) or on an instance (such as C().f()). The instance is ignored except for its class.

You can think of static methods as functions, except they are attached to classes. These classes can either be instatiated or uninstantiated.

In [ ]:
class Square(Shape):
    """Square implementation of Shape."""
    
    def __init__(self, **kwargs):
        self.x = kwargs.get('side', 1)
        self.y = self.x
    
    def __str__(self):
        return "{0} ({1}x{1})".format(self.__class__.__name__, self.x)

    @staticmethod
    def help(*args, **kwargs):
        print("Help for Square class: Accepts side and int value as keyword argument.")
        return args, kwargs

    @classmethod
    def side(cls, side):
        return cls(side=side)

    @classmethod
    def side(cls, side):
        return cls(side=side)

    @property
    def X(self):
        """Public X."""
        return self.x

#    @X.setter
#    def X(self, value):
#        self.x = value

    @X.deleter
    def X(self):
        del self.x

Square.help()

### [Class methods](https://docs.python.org/3.5/library/functions.html#classmethod)

> A class method receives the class as implicit first argument, just like an instance method receives the instance. It can be called either on the class (such as Class.method()) or on an instance (such as C().f()). The instance is ignored except for its class. If a class method is called for a derived class, the derived class object is passed as the implied first argument.

Our `Square` class defines a `side()` method that accepts a `cls` (for class, but we can't use a keyword) argument and is decorated with `@classmethod`. It retuns an instance of the current class, passing the argument to it with the keyword `side`. Our class `ThreeDimensionSquare` inherits this from `Square`. Let's see how it looks like when used:

In [ ]:
s = Square.side(3)
t = ThreeDimensionSquare.side(2)
print(s, t)
s = s.side(2)
print(s)

### [Properties](https://docs.python.org/3.5/library/functions.html#property)

Read-only properties can be created with te `@property` decorator. This decorator is used on methods. To access the property, the method is not called. Note that the values used within these methods can still be accessed normally.

Properties can also have setters and deleters making them editable like other attributes. Uncomment the setter to enable editing the property.

In [ ]:
s = Square.side(3)
print(s.X)
s.X = 4  # this will raise an error if the setter is commented
s.area()

To create setters and deleters, use the property name as decorator, followed by `.setter` or `.deleter`. These should be defined after the `@property` or else you'll encounter a `NameError`. For the `.setter` decorator, the method should accept a value.

In [ ]:
# Try it out!






